Coloreado de Grafos

In [154]:
import random
import pandas as pd
import numpy as np

In [388]:
import random
import pandas as pd
import numpy as np

# Matriz
E = [(1,2),(1,4),(1,16),
    (2,1),(2,3),(2,4),(2,5),(2,6),(2,7),
    (3,2),(3,7),(3,11),(3,19),
    (4,1),(4,2),(4,5),(4,8),(4,9),
    (5,2),(5,4),(5,6),(5,9),(5,10),
    (6,2),(6,5),(6,7),(6,11),(6,14),
    (7,2),(7,3),(7,6),(7,11),
    (8,4),(8,9),(8,12),(8,16),
    (9,4),(9,5),(9,8),(9,10),(9,12),(9,13),(9,15),
    (10,5),(10,9),(10,13),(10,14),
    (11,3),(11,6),(11,7),(11,14),(11,19),
    (12,8),(12,9),(12,15),(12,16),
    (13,9),(13,10),(13,14),(13,15),(13,17),(13,18),
    (14,6),(14,10),(14,11),(14,13),(14,18),(14,19),
    (15,9),(15,12),(15,13),(15,16),(15,17),
    (16,1),(16,8),(16,12),(16,15),(16,17),
    (17,13),(17,15),(17,16),(17,18),(17,19),
    (18,13),(18,14),(18,17),(18,19),
    (19,3),(19,11),(19,14),(19,17),(19,18)]

df = np.zeros((19, 19))
for row, col in E:
    df[row-1, col-1] = 1
df = pd.DataFrame(df, columns=range(1, 20), index=range(1, 20))
df

# Parámetros del algoritmo genético
population_size = 50
num_genes = 19
max_generations = 70
crossover_probability = 0.8
mutation_probability = 0.1
elitism = True #Criterio de reemplazo: Elitismo. Se mantiene al mejor individuo de la generación anterior en la nueva generación.

class Agent:
    def __init__(self, df, num_genes=19):
        self.df = df
        self.num_genes = num_genes
        self.random_initialization()
        self.fitness = self.calc_fitness()

    def random_initialization(self):
        #Criterio de inicialización: Genera un cromosoma aleatorio y asegura factibilidad.
        self.chromosome = [random.choice([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]) for _ in range(self.num_genes)]
        self.create_pairs()
        while not self.check_factibility():
            self.chromosome = [random.choice([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]) for _ in range(self.num_genes)]
            self.create_pairs()

    def check_factibility(self):
        #Función de factibilidad: Verifica que dos nodos conectados no tengan el mismo valor.
        for pair in self.pairs:
            connections = self.df.loc[pair[0]][self.df.loc[pair[0]] == 1].index.tolist()
            for connection in connections:
                if pair[1] == self.pairs[connection-1][1]:
                    return False
                else:
                    continue
        return True
                

    def create_pairs(self):
        #Create a list of position and value pairs
        self.pairs = [[i+1, self.chromosome[i]] for i in range(len(self.chromosome))]



    def calc_fitness(self):
        #Función fitness: Revisa la cantidad de valores distintos en el cromosoma.
        return -len(set(self.chromosome))
        

    def mutate(self):
        #Mutación: Realiza una mutación en un gen del cromosoma.
        idx = random.randint(0, self.num_genes - 1)
        self.chromosome[idx] = random.randint(1,19)  # Cambio 0 a 1 o 1 a 0
        while not self.check_factibility():
            idx = random.randint(0, self.num_genes - 1)
            self.chromosome[idx] = random.randint(1,19)
        self.fitness = self.calc_fitness()

    def crossover(self, partner):
        #Cruce: Realiza un cruce de un punto entre dos cromosomas.
        if random.random() < crossover_probability:
            crossover_point = random.randint(1, self.num_genes - 2)
            child1_chromosome = self.chromosome[:crossover_point] + partner.chromosome[crossover_point:]
            child2_chromosome = partner.chromosome[:crossover_point] + self.chromosome[crossover_point:]

            child1 = Agent(self.df, self.num_genes)
            child2 = Agent(self.df, self.num_genes)
            child1.chromosome = child1_chromosome
            child2.chromosome = child2_chromosome

            if child1.check_factibility():
                child1.fitness = child1.calc_fitness()
            if child2.check_factibility():
                child2.fitness = child2.calc_fitness()

            return child1, child2
        else:
            return self, partner

def selection(population):
    #Criterio de selección: Selección por torneo.
    selected = []
    for _ in range(len(population)):
        ind1 = random.choice(population)
        ind2 = random.choice(population)
        selected.append(ind1 if ind1.fitness > ind2.fitness else ind2)
    return selected

def replacement(old_population, new_population):
    #Criterio de reemplazo: Elitismo.
    if elitism:
        combined = old_population + new_population
        combined.sort(key=lambda x: x.fitness, reverse=True)
        return combined[:population_size]
    else:
        return new_population



In [379]:
#Esto checa si en serio si las soluciones son factibles :)
yo = Agent(df)
print(yo.chromosome)
#Add those values in a list
for pair in yo.pairs:
    connections = df.loc[pair[0]][df.loc[pair[0]] == 1].index.tolist()
    for connection in connections:
        if pair[1] == yo.pairs[connection-1][1]:
            print('Repeated color:', pair[0], connection)
            print('Color:', pair[1])
    

[2, 5, 17, 1, 12, 4, 8, 16, 6, 3, 1, 9, 16, 5, 10, 5, 18, 13, 3]


In [380]:
#Get unique values from the chromosome
unique = []
for i in yo.chromosome:
    if i not in unique:
        unique.append(i)
print(len(unique))

14


In [384]:
yo.fitness

-14

In [389]:
# Inicializar población
population = [Agent(df, num_genes) for _ in range(population_size)]

# Algoritmo Genético
for generation in range(max_generations):
    new_population = []

    selected = selection(population)

    # Cruce
    for i in range(0, population_size, 2):
        parent1 = selected[i]
        parent2 = selected[i + 1]
        child1, child2 = parent1.crossover(parent2)
        new_population.extend([child1, child2])

    # Mutación
    for individual in new_population:
        if random.random() < mutation_probability:
            individual.mutate()

    # Reemplazo
    population = replacement(population, new_population)

    # Criterio de paro: Si se llega al número máximo de generaciones
    best_fitness = max([agent.fitness for agent in population])
    print(f"Generation {generation}: Best Fitness: {best_fitness}")

# Resultado final
best_agent = max(population, key=lambda x: x.fitness)
print("Minimun number of colors:")
print("Colors:" , -best_fitness)
print("Chromosome:", best_agent.chromosome)

Generation 0: Best Fitness: -9
Generation 1: Best Fitness: -9
Generation 2: Best Fitness: -9
Generation 3: Best Fitness: -9
Generation 4: Best Fitness: -9
Generation 5: Best Fitness: -8
Generation 6: Best Fitness: -8
Generation 7: Best Fitness: -8
Generation 8: Best Fitness: -8
Generation 9: Best Fitness: -8
Generation 10: Best Fitness: -8
Generation 11: Best Fitness: -8
Generation 12: Best Fitness: -8
Generation 13: Best Fitness: -8
Generation 14: Best Fitness: -8
Generation 15: Best Fitness: -7
Generation 16: Best Fitness: -7
Generation 17: Best Fitness: -7
Generation 18: Best Fitness: -7
Generation 19: Best Fitness: -7
Generation 20: Best Fitness: -7
Generation 21: Best Fitness: -7
Generation 22: Best Fitness: -7
Generation 23: Best Fitness: -7
Generation 24: Best Fitness: -7
Generation 25: Best Fitness: -7
Generation 26: Best Fitness: -7
Generation 27: Best Fitness: -7
Generation 28: Best Fitness: -6
Generation 29: Best Fitness: -6
Generation 30: Best Fitness: -6
Generation 31: Bes